___
### Research making frequent trades where you are essentially:
1. Trading a binary option around a specific price, by 
2. Trading a call/put spread with a very narrow strike range

#### Usage:
Just run all the cells 
* From Menu Bar: `Kernal->Restart & Run All`
___

In [1]:

# import glob
# basic imports
import pandas as pd
import numpy as np
import datetime

# plotting imports
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin
from IPython import display

# looping imports
from tqdm import tqdm,tqdm_notebook

# vol analysis imports
from py_vollib import black

# extracting zip files from urls and websites imports
import zipfile
import io
import re
import requests
import urllib

BEG_RUN_TIME = datetime.datetime.now()



2020-11-18 10:24:33,466 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


___
#### Conversion and helper methods for yyyymmdd integer dates
___

In [2]:
def dt_to_yyyymmdd(datetime_value):
    '''
    convert datetime.datetime object to integer yyyymmdd, 
       like datetime.datetime(2020,11,16) to 20201116
    '''
    y = int(datetime_value.year)
    m = int(datetime_value.month)
    d = int(datetime_value.day)
    return y*100*100 + m*100 + d

def yyyymmdd_to_dt(yyyymmdd):
    '''
    convert integer (or str) of yyyymmdd to a datetime.dateime object
      like 20201116 to datetime.datetime(2020,11,16)
    (The new datetime object will be Timezone naive)
    '''
    y = int(str(yyyymmdd)[0:4])
    m = int(str(yyyymmdd)[4:6])
    d = int(str(yyyymmdd)[6:8])
    return datetime.datetime(y,m,d)

def yyyymmdd_diff(yyyymmdd_low,yyyymmdd_high):
    '''
    Subtract to yyyymmdd dates
    '''
    dt_low = yyyymmdd_to_dt(yyyymmdd_low)
    dt_high = yyyymmdd_to_dt(yyyymmdd_high)
    return (dt_high-dt_low).days

def sub_days_from_yyyymmdd(yyyymmdd,days):
    '''
    Subtract days from a yyyymmdd date
    '''
    d = yyyymmdd_to_dt(yyyymmdd)
    d2 = d - datetime.timedelta(days)
    return dt_to_yyyymmdd(d2)

def yyyymmdd_dayofweek(yyyymmdd):
    '''
    Get the day of week of a yyyymdd daste
    '''
    return yyyymmdd_to_dt(yyyymmdd).weekday()

___
#### Plotly ploting helpers
___

In [3]:
def figure_crosshairs(fig):
    ''' add crosshairs to plotly_plot figure
    '''
    fig['layout'].hovermode='x'
    fig['layout'].yaxis.showspikes=True
    fig['layout'].xaxis.showspikes=True
    fig['layout'].yaxis.spikemode="toaxis+across"
    fig['layout'].xaxis.spikemode="toaxis+across"
    fig['layout'].yaxis.spikedash="solid"
    fig['layout'].xaxis.spikedash="solid"
    fig['layout'].yaxis.spikethickness=1
    fig['layout'].xaxis.spikethickness=1
    fig['layout'].spikedistance=1000
    return fig


def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,width=800,height=400,
                number_of_ticks_display=20,
                yaxis2_cols=None,
                x_value_labels=None,
                modebar_orientation='v',modebar_color='grey',
                legend_x=None,legend_y=None,
                title_y_pos = 0.9,
                title_x_pos = 0.5):
    '''
    Plot and x/y graph
    '''
    
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    tdtext = tdvals
    if x_value_labels is not None:
        tdtext = [x_value_labels[i] for i in tdvals]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout

    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdtext,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        autosize=True,
#         autosize=False,
#         width=width,
#         height=height,
        margin=Margin(
            b=100
        ),
        modebar={'orientation': modebar_orientation,'bgcolor':modebar_color}
    )

    fig = go.Figure(data=data,layout=layout)
    fig.update_layout(
        title={
            'text': plot_title,
            'y':title_y_pos,
            'x':title_x_pos,
            'xanchor': 'center',
            'yanchor': 'top'})
    if (legend_x is not None) and (legend_y is not None):
        fig.update_layout(legend=dict(x=legend_x, y=legend_y))
    fig = figure_crosshairs(fig)
    return fig


def plotly_shaded_rectangles(beg_end_date_tuple_list,fig):
    '''
    Add shaded rectanges that highlight parts of an x/y graph
    '''
    ld_shapes = []
    for beg_end_date_tuple in beg_end_date_tuple_list:
        ld_beg = beg_end_date_tuple[0]
        ld_end = beg_end_date_tuple[1]
        ld_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=ld_beg[i],
            y0=0,
            x1=ld_end[i],
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
        ld_shapes.append(ld_shape)

    fig.update_layout(shapes=ld_shapes)
    return fig

In [4]:
def filter_df(df,conditions):
    '''
    Filter a DataFrame from multiple conditions
    See Example Below in next cell
    '''
    c = conditions
    if type(conditions)==list:
        c = np.array(c)
    if type(c)not in [np.array,np.ndarray]:
        raise ValueError(f"invalid type for argument conditions: {type(conditions)}")
    c = [all(a) for a in c.T]
    return df[c]


In [5]:
df_test = pd.DataFrame({'a':np.arange(1,11),'b':np.arange(21,31),'c':np.arange(31,41)})
display.display(df_test)
conditions = [
    df_test.a.isin([1,3,5]),
    (df_test.b==23) | (df_test.b==25)
]
display.display(filter_df(df_test,conditions))

,a,b,c
0,1,21,31
1,2,22,32
2,3,23,33
3,4,24,34
4,5,25,35
5,6,26,36
6,7,27,37
7,8,28,38
8,9,29,39
9,10,30,40


,a,b,c
2,3,23,33
4,5,25,35


In [6]:
def download_extract_zip(url):
    """
    Download a ZIP file and extract its contents in memory
    yields (filename, file-like object) pairs
    """
#     response = requests.get(url)
    mysock = urllib.request.urlopen(url)
    memfile = io.BytesIO(mysock.read())
    with zipfile.ZipFile(memfile, 'r') as thezip:
        d =  {
            name: pd.read_csv(io.BytesIO(thezip.read(name)))
            for name in thezip.namelist()}
        return list(d.values())[0]


___
#### Download settlement files from ftp://ftp.cmegroup.com/settle
___

#### Old CME exchange contracts

In [7]:
def get_settle_from_cme_ftp(settle_yyyymmdd,exch_to_fetch='cme'):
    extra_dot = '.' if len(str(settle_yyyymmdd))>0 else ''
    csv_url = f'ftp://ftp.cmegroup.com/settle/{exch_to_fetch}.settle{extra_dot}{str(settle_yyyymmdd)}.s.csv'
    df = download_extract_zip(csv_url+'.zip')
    return df
    

In [8]:
yyyymmdd_to_fetch = 20201116
exch_to_fetch = 'cme' # could be one of cme, nymex, comex, cbt
df_cme = get_settle_from_cme_ftp(yyyymmdd_to_fetch,exch_to_fetch=exch_to_fetch)

In [9]:
df_cme.columns.values

array(['BizDt', 'Sym', 'ID', 'StrkPx', 'SecTyp', 'MMY', 'MatDt',
       'PutCall', 'Exch', 'Desc', 'LastTrdDt', 'BidPrice', 'OpeningPrice',
       'SettlePrice', 'SettleDelta', 'HighLimit', 'LowLimit',
       'DHighPrice', 'DLowPrice', 'HighBid', 'LowBid', 'PrevDayVol',
       'PrevDayOI', 'FixingPrice', 'UndlyExch', 'UndlyID', 'UndlySecTyp',
       'UndlyMMY', 'BankBusDay'], dtype=object)

#### NYMEX Contracts

In [10]:
yyyymmdd_to_fetch='20201113'
exch_to_fetch = 'nymex' # could be one of cme, nymex, comex, cbt
df_nym = get_settle_from_cme_ftp(yyyymmdd_to_fetch,exch_to_fetch=exch_to_fetch)


In [11]:
vv = [v[0]+"_"+str(v[1]) for v in df_nym[df_nym.Sym.str.startswith('LO')][['Sym','MMY']].values]
np.sort(np.unique(vv))

array(['LO1_202012', 'LO2_202011', 'LO3_202011', 'LO4_202011',
       'LO_202012', 'LO_202101', 'LO_202102', 'LO_202103', 'LO_202104',
       'LO_202105', 'LO_202106', 'LO_202107', 'LO_202108', 'LO_202109',
       'LO_202110', 'LO_202111', 'LO_202112', 'LO_202201', 'LO_202202',
       'LO_202203', 'LO_202206', 'LO_202207', 'LO_202208', 'LO_202209',
       'LO_202210', 'LO_202212', 'LO_202306', 'LO_202312'], dtype='<U10')

In [12]:
df_nym[df_nym.Sym.str.startswith('LO')].Sym.unique()

array(['LO', 'LO1', 'LO2', 'LO3', 'LO4'], dtype=object)

#### Comex contracts

In [13]:
yyyymmdd_to_fetch='20201116'
exch_to_fetch = 'comex' # could be one of cme, nymex, comex, cbt
df_comex = get_settle_from_cme_ftp(yyyymmdd_to_fetch,exch_to_fetch=exch_to_fetch)


In [14]:
np.unique([v[0]+"_"+str(v[1]) for v in df_comex[df_comex.Sym.str.startswith('OG')][['Sym','MMY']].values])

array(['OG1_202012', 'OG2_202012', 'OG3_202011', 'OG4_202011',
       'OG_202012', 'OG_202101', 'OG_202102', 'OG_202103', 'OG_202104',
       'OG_202105', 'OG_202106', 'OG_202107', 'OG_202108', 'OG_202109',
       'OG_202110', 'OG_202111', 'OG_202112', 'OG_202201', 'OG_202202',
       'OG_202203', 'OG_202204', 'OG_202205', 'OG_202206', 'OG_202207',
       'OG_202212', 'OG_202306', 'OG_202312', 'OG_202406', 'OG_202412',
       'OG_202506', 'OG_202512', 'OG_202606'], dtype='<U10')

In [15]:

comex_filters = [
    df_comex.MatDt.str.contains('2020-11'),
    df_comex.Sym.str.startswith('OG')
]

np.sort(filter_df(df_comex,comex_filters).Sym.unique())

array(['OG', 'OG3', 'OG4'], dtype=object)

___
#### Use filter_df to find weekly ES options that are near the money for:
1. The trade date of `20201116` (which is the date on the zip file)
2. The date in the column df_cme.MatDt of `2020-11-18` (this date is the string version of a datetime.datetime object
___


In [16]:
# date to filter the MatDt column of df_cme
filter_date='2020-11-18'

# get the settlement date of the nearby ES contract
futures_conditions  =  [
    df_cme.Sym=='ES',
    df_cme.SecTyp=='FUT'
]
df_cme_es_fut = filter_df(df_cme,futures_conditions)
settle = df_cme_es_fut.iloc[0].SettlePrice

# get the revelant options that are close to the futures price, and are weekly options
options_conditions = [
    (df_cme.Sym.astype(str).str.contains('E[0-9][A-Z]')).values,
    (df_cme.MatDt.astype(str).str.contains(str(filter_date))).values,
    (df_cme.StrkPx>=settle-25).values,
    (df_cme.StrkPx<=settle+25).values,
    (df_cme.PutCall==1).values,
]
filter_df(df_cme,options_conditions)

,BizDt,Sym,ID,StrkPx,SecTyp,MMY,MatDt,PutCall,Exch,Desc,...,HighBid,LowBid,PrevDayVol,PrevDayOI,FixingPrice,UndlyExch,UndlyID,UndlySecTyp,UndlyMMY,BankBusDay
87838,2020-11-16,E3C,E3C,3600.0,OOF,202011,2020-11-18,1.0,CME,NaN,...,NaN,22.00,343,311,NaN,CME,ES,FUT,202012.0,NaN
87840,2020-11-16,E3C,E3C,3605.0,OOF,202011,2020-11-18,1.0,CME,NaN,...,44.50,19.25,82,212,NaN,CME,ES,FUT,202012.0,NaN
87842,2020-11-16,E3C,E3C,3610.0,OOF,202011,2020-11-18,1.0,CME,NaN,...,NaN,17.00,410,255,NaN,CME,ES,FUT,202012.0,NaN
87844,2020-11-16,E3C,E3C,3615.0,OOF,202011,2020-11-18,1.0,CME,NaN,...,37.50,14.75,36,149,NaN,CME,ES,FUT,202012.0,NaN
87846,2020-11-16,E3C,E3C,3620.0,OOF,202011,2020-11-18,1.0,CME,NaN,...,NaN,12.75,365,569,NaN,CME,ES,FUT,202012.0,NaN
87848,2020-11-16,E3C,E3C,3625.0,OOF,202011,2020-11-18,1.0,CME,NaN,...,31.50,11.00,123,549,NaN,CME,ES,FUT,202012.0,NaN
87850,2020-11-16,E3C,E3C,3630.0,OOF,202011,2020-11-18,1.0,CME,NaN,...,28.50,9.50,86,146,NaN,CME,ES,FUT,202012.0,NaN
87852,2020-11-16,E3C,E3C,3635.0,OOF,202011,2020-11-18,1.0,CME,NaN,...,25.75,NaN,114,175,NaN,CME,ES,FUT,202012.0,NaN
87854,2020-11-16,E3C,E3C,3640.0,OOF,202011,2020-11-18,1.0,CME,NaN,...,23.25,NaN,187,309,NaN,CME,ES,FUT,202012.0,NaN
87856,2020-11-16,E3C,E3C,3645.0,OOF,202011,2020-11-18,1.0,CME,NaN,...,NaN,5.50,183,143,NaN,CME,ES,FUT,202012.0,NaN


___
### Return a dictionary which has information relating to a ES call spreads for strikes very close to an ES futures settlement

This method returns the prices of options on Weekly ES contract:
* E1A, E2A, E3A, E4A, E1C, E2C, E3C, E4C
___


In [17]:
def process_call_spread(df_cme,filter_date,pc=1):
    futures_conditions  =  np.array([
        df_cme.Sym=='ES',
        df_cme.SecTyp=='FUT'
    ])
    df_cme_es_fut = filter_df(df_cme,futures_conditions)[['MMY','SettlePrice']]
    settle = df_cme_es_fut.iloc[0].SettlePrice
    mmy =  df_cme_es_fut.iloc[0].MMY

    options_conditions = np.array([
        (df_cme.Sym.astype(str).str.contains('E[0-9][A-Z]')).values,
        (df_cme.MatDt.astype(str).str.contains(filter_date)).values,
        (df_cme.StrkPx>=settle-25).values,
        (df_cme.StrkPx<=settle+25).values,
        (df_cme.PutCall==1).values,
    ])
    df_cme_es_calls = strike_prices = filter_df(df_cme,options_conditions)[['Sym','StrkPx','SettlePrice','PutCall']]
    df_cme_es_calls['abs_diff_from_settle'] = (df_cme_es_calls.StrkPx - settle).abs()
    df_cme_es_calls = df_cme_es_calls.sort_values('abs_diff_from_settle').iloc[0:5].sort_values('StrkPx')
    call1 = df_cme_es_calls.iloc[0] # in the money call
    call2 = df_cme_es_calls.iloc[2] # out of the money call
    call_strike_middle = df_cme_es_calls.iloc[1].StrkPx
    call_spread_price = call1.SettlePrice - call2.SettlePrice
    call1_itms = settle - call1.StrkPx
    return {
        'df_fut':df_cme_es_fut,
        'df_calls':df_cme_es_calls,
        'settle':settle,
        'mmy':mmy,
        'call_spread_price':call_spread_price,
        'call1_itms':call1_itms
    }



In [18]:
results = process_call_spread(df_cme,'2020-11-18')

In [19]:
results['mmy'],results['settle'],results['call_spread_price'],results['call1_itms']

(202012.0, 3623.0, 5.899999999999999, 8.0)

In [20]:
results['df_calls']

,Sym,StrkPx,SettlePrice,PutCall,abs_diff_from_settle
87844,E3C,3615.0,23.7,1.0,8.0
87846,E3C,3620.0,20.6,1.0,3.0
87848,E3C,3625.0,17.8,1.0,2.0
87850,E3C,3630.0,15.2,1.0,7.0
87852,E3C,3635.0,12.8,1.0,12.0


___
### Run Black Scholes on the ES 10 tick call spreads around the money for:
1. A range of At The Money values
2. A range of volatilities
___

In [21]:
#!pip install py_vollib
import numpy
from py_vollib import black
flag="c"
for F in range(1000,4000,500):
    for sigma in [round(v,3) for v in np.arange(.1,.3,.05)]:
        
        K=F-5
        t=1/365
        rate=.015
        skew=10/F
        c1 = black.black(flag, F, K, t, rate, sigma+skew)
        c2 = black.black(flag, F, K+10, t, rate, sigma-skew)
        print(f'Futures: {F}  vol: {sigma}  callspread: {c1-c2}')

Futures: 1000  vol: 0.1  callspread: 5.256961555618547
Futures: 1000  vol: 0.15  callspread: 5.327710031144061
Futures: 1000  vol: 0.2  callspread: 5.353660614717189
Futures: 1000  vol: 0.25  callspread: 5.363695660252613
Futures: 1500  vol: 0.1  callspread: 5.331972144507231
Futures: 1500  vol: 0.15  callspread: 5.36704216089039
Futures: 1500  vol: 0.2  callspread: 5.376890954137092
Futures: 1500  vol: 0.25  callspread: 5.37879202503154
Futures: 2000  vol: 0.1  callspread: 5.362978396170131
Futures: 2000  vol: 0.15  callspread: 5.381855457677941
Futures: 2000  vol: 0.2  callspread: 5.385362577171727
Futures: 2000  vol: 0.25  callspread: 5.384216269860559
Futures: 2500  vol: 0.1  callspread: 5.378260341460183
Futures: 2500  vol: 0.15  callspread: 5.388904679255704
Futures: 2500  vol: 0.2  callspread: 5.389345208916264
Futures: 2500  vol: 0.25  callspread: 5.386752011718212
Futures: 3000  vol: 0.1  callspread: 5.3868192256444
Futures: 3000  vol: 0.15  callspread: 5.392785989967667
Futur

___
### Run The buy callspread strategy
___


#### Method to create contract numbers for each settle_date in the Futures DataFrame, where:
1. `1` is the number for the most nearby contract
  * if the contracts are `['ESU10', 'ESZ10', 'ESH11', 'ESM11', 'ESU11']`, then contract 1 is `ESU10`
2. `n` is the last contract
  * if the contracts are `['ESU10', 'ESZ10', 'ESH11', 'ESM11', 'ESU11']`, then contract n is `ESU11`


In [22]:
def apply_contract_numbers(df):
    df2 = df.copy()
    df2['sym2'] = df2.symbol.str.slice(0,2) + df2.symbol.str.slice(-2) + df2.symbol.str.slice(-3,-2)
    df2_gb = df2.sort_values(['settle_date','sym2']).groupby('settle_date',as_index=False)
    def _get_cnum(df_single_day_sorted,alt_symbol_col='sym2'):
        df_single_day_sorted['contract_num'] = list(range(len(df_single_day_sorted)))
        return df_single_day_sorted
    df2 = df2_gb.apply(_get_cnum)
    return df2



#### Read a previously saved csv file of Futures prices from `20100901` to `20200908`

In [23]:
df_es_db = pd.read_csv('./temp_folder/df_es_db.csv')
df_es_db2 = apply_contract_numbers(df_es_db)
df_es_db_front = df_es_db2[df_es_db2.contract_num==0]
# show contracts for the first settle_date in the csv file
df_es_db2[df_es_db2.settle_date == df_es_db2.settle_date.unique()[0]].symbol.values

array(['ESU10', 'ESZ10', 'ESH11', 'ESM11', 'ESU11', 'ESZ99'], dtype=object)

#### Filter df_es_db2 so that it only contains prices for the current front month ES contract

In [24]:
df_es_db_front2 = df_es_db_front.copy()
df_es_db_front2['prev_symbol'] = df_es_db_front2.symbol.shift()
df_es_db_front2['same_contract'] = df_es_db_front2.symbol==df_es_db_front2.prev_symbol
df_es_db_front2['pct_chg'] = df_es_db_front2.close.pct_change()*df_es_db_front2.same_contract
df_es_db_front2 = df_es_db_front2[~df_es_db_front2.pct_chg.isna()]
df_es_db_front2



,symbol,settle_date,contract_num,open,high,low,close,adj_close,volume,open_interest,expiry,exp_yyyymmdd,dte,sym2,prev_symbol,same_contract,pct_chg
7,ESU10,20100902,0,1081.50,1090.50,1076.50,1089.50,1089.50,2355074,2901867,2010-09-17,20100917,15,ES10U,ESU10,True,0.007164
13,ESU10,20100903,0,1089.75,1104.50,1086.25,1103.50,1103.50,1601762,2875870,2010-09-17,20100917,14,ES10U,ESU10,True,0.012850
19,ESU10,20100907,0,1103.75,1107.25,1089.75,1091.25,1091.25,2017072,2850566,2010-09-17,20100917,10,ES10U,ESU10,True,-0.011101
25,ESU10,20100909,0,1099.25,1112.00,1096.75,1102.50,1102.50,2001331,2602936,2010-09-17,20100917,8,ES10U,ESU10,True,0.010309
31,ESU10,20100910,0,1103.00,1110.50,1100.50,1109.75,1109.75,1612227,2246071,2010-09-17,20100917,7,ES10U,ESU10,True,0.006576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15217,ESU20,20200901,0,3493.25,3530.00,3484.25,3527.00,3527.00,1439600,2688708,2020-09-18,20200918,17,ES20U,ESU20,True,0.008002
15223,ESU20,20200902,0,3529.00,3587.00,3526.25,3579.25,3579.25,1283622,2696614,2020-09-18,20200918,16,ES20U,ESU20,True,0.014814
15229,ESU20,20200903,0,3578.75,3586.50,3424.50,3461.50,3461.50,1853078,2680985,2020-09-18,20200918,15,ES20U,ESU20,True,-0.032898
15235,ESU20,20200904,0,3454.75,3484.25,3347.75,3417.50,3417.50,2872337,2665365,2020-09-18,20200918,14,ES20U,ESU20,True,-0.012711


___
### Run the call spread strategy, but on call spreads that are 5 ticks wide
___

In [25]:
thresh = 0.000
strike_diff = 5
call_spread_price = 2.7



df_es_db_front2_up = df_es_db_front2[df_es_db_front2.pct_chg>thresh]
df_es_db_front2_down = df_es_db_front2[df_es_db_front2.pct_chg<thresh]
profit = len(df_es_db_front2_up)*(strike_diff-call_spread_price) - len(df_es_db_front2_down)*call_spread_price
up_days = len(df_es_db_front2_up)
down_days = len(df_es_db_front2_down)
all_days = up_days + down_days
{
    'all_days':all_days,
    'up_days':up_days,
    'down_days':down_days,
    'up_days/all_days':up_days/all_days,
    'down_days/all_days':down_days/all_days,
    'profit':profit,
    'per_trade_avg_profit':profit/all_days,
    'mean_pct_change':df_es_db_front2.pct_chg.mean(),
    'mean_uppct_change':df_es_db_front2_up.pct_chg.mean(),
    'mean_downpct_change':df_es_db_front2_down.pct_chg.mean()

}
    

{'all_days': 2449,
 'up_days': 1375,
 'down_days': 1074,
 'up_days/all_days': 0.5614536545528788,
 'down_days/all_days': 0.4385463454471213,
 'profit': 262.69999999999936,
 'per_trade_avg_profit': 0.10726827276439337,
 'mean_pct_change': 0.0006315430617851898,
 'mean_uppct_change': 0.006752222153110523,
 'mean_downpct_change': -0.007161007742684291}

In [31]:
END_RUN_TIME=datetime.datetime.now()
print(f"start:{BEG_RUN_TIME}") 
print(f"end:{END_RUN_TIME}")
print(f"duration:{END_RUN_TIME-BEG_RUN_TIME}")

start:2020-11-18 10:24:33.906782
end:2020-11-18 10:27:45.482368
duration:0:03:11.575586


## END